Importamos las librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from datetime import timezone

Creamos las variables que guardan las rutas de los archivos fuente de datos para lectura y grabación de los mismos

In [ ]:
# Inicializar entorno de rutas en google drive para carga de archivos 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Nombre del directorio de datos para rutas de ejecucion en ENTORNO LOCAL
PATH_read_clima = "clima"
PATH_read_datos = "Diarios_2013-2019"
PATH_write = "datoscsv"
pathclima = os.path.join(PATH_read_clima)
pathdatos = os.path.join(PATH_read_datos)en
pathgraba = os.path.join(PATH_write)

In [ ]:
# cargar dataframes Version Local
'''
maindf = pd.read_csv(pathdatos+"/Diarios_2013-2019.csv",sep=";",decimal=".")
climadf = pd.read_csv(pathclima +'/CLIMA_2013-2019.csv', delimiter=',', decimal=".")
'''
# cargar dataframes Version Colab en Drive
maindf = pd.read_csv("/content/drive/MyDrive/IABD_Proyecto/20220515_IABD_PRJ_Dev_MIKEL/Diarios_2013-2019/Diarios_2013-2019.csv",sep=";",decimal=".")
climadf = pd.read_csv("/content/drive/MyDrive/IABD_Proyecto/20220515_IABD_PRJ_Dev_MIKEL/clima/CLIMA_2013-2019.csv", delimiter=',', decimal=".")

- Aseguramos que el tipo de datos en campo fecha sea `datetime`
- Aseguramos el uso de la puntuación decimal y la conversión de tipos a numérico de todas aquellas observaciones que sean de esa naturaleza

In [ ]:
# Convertir la fecha a formato datetime
climadf['Fecha'] = pd.to_datetime(climadf['Fecha'], format="%d/%m/%y")
columclima = climadf.columns.to_list()
borra3 = 'Fecha'
borra = 'HoraRacha'
columclima = list(filter((borra3).__ne__,columclima))
columclima = list(filter((borra).__ne__,columclima))
for colcli in columclima:
    climadf[colcli] = climadf[colcli].replace(',','.', regex=True)
    climadf[colcli] = climadf[colcli].apply(pd.to_numeric)

- Eliminamos las columnas cuyo porcentaje de Nan , nulos o similares esta por encima del 90%
- Aseguramos que el tipo de datos en campos de texto y fecha sea el que corresponde

In [ ]:
maindf.drop([ 'BaP','BbFA','BjFA','BkFA','PST', 'UVB','H2S', 'HNM', 'IcdP', 'HCT','HMN', 'FA', 'DahA', 'CH4', 'BaA'] , axis=1, inplace= True)
maindf.drop([ 'NH3','C6H6','C7H8','C8H10','As', 'Cd','Ni', 'Pb'] , axis=1, inplace= True)
# Convertir las cadenas a formato string
maindf['COD_ESTACION'] = maindf['COD_ESTACION'].astype(str)
maindf['NOM_ESTACION'] = maindf['NOM_ESTACION'].astype(str)
# Convertir la fecha a formato datetime
maindf['FECHA'] = pd.to_datetime(maindf['FECHA'], format="%d/%m/%y")

Aseguramos el uso de la puntuación decimal y la conversión de tipos a numérico de todas aquellas observaciones que sean de esa naturaleza

In [ ]:
columnas = maindf.columns.to_list()
borra1 = "COD_ESTACION"
borra2 = 'NOM_ESTACION'
borra3 = 'FECHA'
columnas = list(filter((borra1).__ne__,columnas))
columnas = list(filter((borra2).__ne__,columnas))
columnas = list(filter((borra3).__ne__,columnas))

In [ ]:
for col in columnas:
    maindf[col] = maindf[col].replace(',','.', regex=True)
    maindf[col] = maindf[col].apply(pd.to_numeric)

### Fusion de Dataframes de clima y calidad del aire ###
- El campo comun es la fecha.
- Los datos de las columnas de clima se repetiran en todas las fechas coincidentes del dataframe resultante.
- Al tratarse de lecturas de datos climatológicos de toda la ciudad ( dataframe `climadf`), todas las estationes tendrán los mismos datos climáticos , en caso de que originalmente esas observaciones estén vacios en el dataframe `maindf`. 

In [ ]:
df = pd.merge(left=maindf, right=climadf, how='left', left_on='FECHA', right_on='Fecha')

Eliminamos las filas correspondientes a la estación Valencia Albufera por motivos geograficos ( mediciones fuera del area metropolitana ) y presencia únicamente en algunos años dentro del rango tratado

In [ ]:
indexNames = df[ df['COD_ESTACION'] == '46250051' ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

En el caso de la temperatura los huecos se rellenarán con la media calculada de las temperaturas máxima y mínima que provienen del dataframe de clima (`climadf`)

`Tmedia = df.loc[:,'TempMax':'TempMin'].mean(axis=1)`

`df.loc[df['Temp'].isna(),'Temp']=Tmedia`

- Eliminamos las columnas repetidas y las que porcentaje de Nan , nulos o similares esta por encima del 80%

In [ ]:
# actualizar columna 'Temp.' con 'TEMP' cuando el campo 'Temp.' no tenga valor
Tmedia = df.loc[:,'TempMax':'TempMin'].mean(axis=1)
df.loc[df['Temp'].isna(),'Temp']=Tmedia
df.loc[df['lluvias'].isna(),'lluvias']=df['Precip']
df.loc[df['VelocVent'].isna(),'VelocVent']=df['Veloc']
df.loc[df['VelocVentMax'].isna(),'VelocVentMax']=df['RachaMax']
df.drop([ 'Precip','Veloc','RachaMax', 'Fecha','HumRel','Pres','RadSol','Ruido','DirecVent'] , axis=1, inplace= True)

Esta operación permite una reducción drástica de los porcentajes de valores NaN , nulos o similares.

Aumentando así la fiabiliad en los pasos posteriores.

In [ ]:
maindf.isnull().sum()/len(maindf)*100

In [ ]:
df.isnull().sum()/len(df)*100

In [ ]:
df.describe().transpose()

Finalmente guardamos el resultado en un fichero csv para futuras tareas.

In [ ]:
# Guarda datos en CSV:
df.to_csv(pathgraba+'/airWeather_2013-2019.csv', header=True, sep=';', index=False)